In [40]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
import pickle as pkl



In [30]:
ds=pd.read_csv("new_cleanesd_data.csv")
ds=ds.loc[:,~ds.columns.str.contains('^Unnamed')]  #removing
ds

,Gender,EthnicGroup,ParentEduc,LunchType,TestPrep,MathScore,ReadingScore,WritingScore
0,female,group B,bachelor's degree,1,none,72,72,74
1,female,group C,some college,1,completed,69,90,88
2,female,group B,master's degree,1,none,90,95,93
3,male,group A,associate's degree,0,none,47,57,44
4,male,group C,some college,1,none,76,78,75
...,...,...,...,...,...,...,...,...
30636,male,group C,some high school,1,none,56,47,51
30637,male,group E,associate's degree,0,none,74,75,72
30638,male,group C,some college,1,none,36,29,27
30639,male,group A,some high school,0,completed,43,34,39


In [31]:
#splitting data into independent and dependent variables
x=ds[['Gender','EthnicGroup','ParentEduc','LunchType','TestPrep']]
y=ds[['MathScore']]

In [32]:
print(len(ds['Gender'].unique()))
print(len(ds['EthnicGroup'].unique()))
print(len(ds['ParentEduc'].unique()))
print(len(ds['LunchType'].unique()))
print(len(ds['TestPrep'].unique()))

2
5
6
2
2


In [33]:
# implement OneHotEncoder

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
ohe=OneHotEncoder()
ohe.fit(x[["Gender","EthnicGroup","ParentEduc","LunchType","TestPrep"]])

OneHotEncoder()

In [34]:
ohe.categories_

[array(['female', 'male'], dtype=object),
 array(['group A', 'group B', 'group C', 'group D', 'group E'],
       dtype=object),
 array(["associate's degree", "bachelor's degree", 'high school',
        "master's degree", 'some college', 'some high school'],
       dtype=object),
 array([0, 1], dtype=int64),
 array(['completed', 'none'], dtype=object)]

In [35]:
ct=make_column_transformer((OneHotEncoder(handle_unknown='ignore',categories=ohe.categories_),["Gender","EthnicGroup","ParentEduc","LunchType","TestPrep"]),remainder='passthrough',force_int_remainder_cols=False,sparse_threshold=0)
ct

ColumnTransformer(force_int_remainder_cols=False, remainder='passthrough',
                  sparse_threshold=0,
                  transformers=[('onehotencoder',
                                 OneHotEncoder(categories=[array(['female', 'male'], dtype=object),
                                                           array(['group A', 'group B', 'group C', 'group D', 'group E'],
      dtype=object),
                                                           array(["associate's degree", "bachelor's degree", 'high school',
       "master's degree", 'some college', 'some high school'],
      dtype=object),
                                                           array([0, 1], dtype=int64),
                                                           array(['completed', 'none'], dtype=object)],
                                               handle_unknown='ignore'),
                                 ['Gender', 'EthnicGroup', 'ParentEduc',
                                  'LunchType', 'TestPrep'])])

In [36]:
#Making pipeline
from sklearn.linear_model import LinearRegression
reg=LinearRegression()


In [37]:
from sklearn.pipeline import make_pipeline
pipe=make_pipeline(ct,reg)
pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(force_int_remainder_cols=False,
                                   remainder='passthrough', sparse_threshold=0,
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['female', 'male'], dtype=object),
                                                                            array(['group A', 'group B', 'group C', 'group D', 'group E'],
      dtype=object),
                                                                            array(["associate's degree", "bachelor's degree", 'high school',
       "master's degree", 'some college', 'some high school'],
      dtype=object),
                                                                            array([0, 1], dtype=int64),
                                                                            array(['completed', 'none'], dtype=object)],
                                                                handle_unknown='ignore'),
                                                  ['Gender', 'EthnicGroup',
                                                   'ParentEduc', 'LunchType',
                                                   'TestPrep'])])),
                ('linearregression', LinearRegression())])

In [38]:
# splitting dataset into training and testing

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
scores=[]
for i in range(0,101):
    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.10,random_state=i)
    pipe.fit(x_train,y_train)
    result=pipe.predict(x_test)
    score=r2_score(y_test,result)
    scores.append(score)

In [41]:
# finding best score
bestindex=np.argmax(scores)
scores[bestindex]

0.3153366806146952

In [42]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.10,random_state=bestindex)
pipe.fit(x_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(force_int_remainder_cols=False,
                                   remainder='passthrough', sparse_threshold=0,
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['female', 'male'], dtype=object),
                                                                            array(['group A', 'group B', 'group C', 'group D', 'group E'],
      dtype=object),
                                                                            array(["associate's degree", "bachelor's degree", 'high school',
       "master's degree", 'some college', 'some high school'],
      dtype=object),
                                                                            array([0, 1], dtype=int64),
                                                                            array(['completed', 'none'], dtype=object)],
                                                                handle_unknown='ignore'),
                                                  ['Gender', 'EthnicGroup',
                                                   'ParentEduc', 'LunchType',
                                                   'TestPrep'])])),
                ('linearregression', LinearRegression())])

In [45]:
# predict for user_input
Gender=input("enter 'Gender:")
EthnicGroup=input("enter EthnicGroup:")

ParentEduc=(input("enter ParentEduc:"))
LunchType=int(input("enter LunchType:"))

TestPrep=input("enter TestPrep:")
columns=["Gender","EthnicGroup","ParentEduc","LunchType","TestPrep"]
myinput=pd.DataFrame(columns=columns,data=[[Gender,EthnicGroup,ParentEduc,LunchType,TestPrep]])
result=pipe.predict(myinput)
print("Mathe score predicted:",abs(round(result[0,0])))

enter 'Gender: male
enter EthnicGroup: group A
enter ParentEduc: high school
enter LunchType: 1
enter TestPrep: none


Mathe score predicted: 66


In [46]:
import pickle as pkl

In [49]:
pkl.dump(pipe,open("student.pkl","wb"))